In [ ]:
import gemini as gemini
import config as config
import bq as bq
import gcs as gcs
import utils as utils

import video_intelligence as gvi
from google.cloud import videointelligence_v1 as vi


config.WORKING_BUCKET = bucket = "video-working-bucket-e2c3"
config.OUTPUT_BUCKET = "video-working-bucket-e2c3"
config.INPUT_BUCKET = "video-input-bucket-e2c3"
config.PROJECT_ID = "media-414316"
config.BQ_DATASET = "video_analytics_5e3d"



import json
import pandas as pd


# Video intelligence API

In [ ]:
from google.cloud import videointelligence_v1 as vi


In [ ]:
# Shot change
file = "fr-FR/SHOT_CHANGE_DETECTION/cdanslair - 1708961447.5183704.json"

data = gcs.read_json_from_gcs(bucket, file)

print(data)
annotation = vi.AnnotateVideoResponse(data)

gvi.splitVideo(data)

In [ ]:
# Explicit content
file = "fr-FR/EXPLICIT_CONTENT_DETECTION/test1 - BEST UPCOMING MOVIES 2024 (Trailers) - 1709023409.1792483.json"
data = gcs.read_json_from_gcs(bucket, file)

print(data)
annotation = vi.AnnotateVideoResponse(data)

#gvi.splitVideo(data)
gvi.storeVideoIntelligenceData(data)

In [ ]:

# TEXT DETECTION

file = "fr-FR/TEXT_DETECTION/test1 - BEST UPCOMING MOVIES 2024 (Trailers) - 1709023409.2954755.json"

data = gcs.read_json_from_gcs(bucket, file)
annotation = vi.AnnotateVideoResponse(data)

#gvi.splitVideo(data)
gvi.storeVideoIntelligenceData(data)

# Gemini

In [ ]:
uri = "gs://video-output-bucket-e2c3/CSA/fr-FR/cdanslair.mp4/chunks - 3 - 7.36 - 9.96.mp4"
name = "CSA/fr-FR/cdanslair.mp4/chunks - 3 - 7.36 - 9.96.mp4"

res = gemini.content_moderation_gemini(uri)
print(f"moderation content done on chunck uri {uri} with res = {res}")

print("save json result in output bucket")
json_file_path = gcs.write_text_to_gcs(config.OUTPUT_BUCKET, utils.replace_extension(name, ".json"), res, "text/json")
print(f"json_file_path = {json_file_path}")

dict= json.loads(res)
dict= dict["csa_rules"]

print("read tags from source uri")
bucketname, video_blobname = gcs.split_gcs_uri(uri)
tags = gcs.read_tags_from_gcs(bucketname, video_blobname)

if tags is None:
    print("no tags found")
else:
    print(f"uri= {uri} - tags = {tags}")
    dict.update(tags)            

dict["uri"] = uri
df = pd.DataFrame([dict])

df